In [2]:
import zipfile as zp
import os
df = zp.ZipFile('WEATHER-5K.zip')
# for i in df.namelist():
#     print(i)
print(len(df.namelist()))
# df.extractall()

5675


In [3]:
files_to_extract = [
    'WEATHER-5K/global_weather_stations/01001099999.csv',
    'WEATHER-5K/meta_info.json',
    'WEATHER-5K/percentile.json'
]
for file in files_to_extract:
    df.extract(file)
    print(f'{file} extracted')
df.close()

WEATHER-5K/global_weather_stations/01001099999.csv extracted
WEATHER-5K/meta_info.json extracted
WEATHER-5K/percentile.json extracted


In [ ]:
import pandas as pd
import os
# 读取WEATHER-5K/global_weather_stations中以csv结尾的文件
folder_path = "WEATHER-5K/global_weather_stations"
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
# mask代表数据的缺失情况
for file in csv_files:
    # 构造完整的文件路径
    file_path = os.path.join(folder_path, file)
    # 读取 CSV 文件
    df = pd.read_csv(file_path)
    # print(df.shape)
    # print(type(df.iloc[1]['MASK']))
    # print(type(df.iloc[1]['TIME_DIFF']))
    # condition1 = (df['TIME_DIFF'] == 0) & (df['MASK'] != "[1 1 1 1 1]")
    # condition2 = (df['TIME_DIFF'] == 999999.0) & (df['MASK'] != "[0 0 0 0 0]")
    # print(df[condition2].shape)
    # filtered_df = df[condition1 | condition2]
    # print(filtered_df.index)
    # print(df.head(5))
    # break

(0, 10)
(0, 10)
(0, 10)
(0, 10)
(0, 10)
(0, 10)
(0, 10)
(0, 10)
(0, 10)
(0, 10)


存储成npz文件

In [55]:
import os
import argparse
import numpy as np
import pandas as pd

class StandardScaler():
    def __init__(self, mean, std):
        self.mean = mean
        self.std = std

    def transform(self, data):
        return (data - self.mean) / self.std

    def inverse_transform(self, data):
        return (data * self.std) + self.mean

def generate_data_and_idx(data, x_offsets, y_offsets, add_time_of_day, add_day_of_week):
    num_samples, num_nodes, input_dim = data.shape
    print('num_samples:', num_samples, 'num_nodes:', num_nodes)

    feature_list = [data]
    
    if add_time_of_day:
        # 假设 data 的索引是时间戳，存储在第一个维度
        time_ind = (np.arange(num_samples) - np.arange(num_samples) // 24) / 24.0
        time_of_day = np.tile(time_ind, [1, num_nodes, 1]).transpose((2, 1, 0))
        feature_list.append(time_of_day)
    
    if add_day_of_week:
        # 假设 data 的索引是时间戳，存储在第一个维度
        dow = np.arange(num_samples) % 7
        dow_tiled = np.tile(dow, [1, num_nodes, 1]).transpose((2, 1, 0))
        day_of_week = dow_tiled / 7.0
        feature_list.append(day_of_week)

    data = np.concatenate(feature_list, axis=-1)
    
    min_t = abs(min(x_offsets))
    max_t = abs(num_samples - abs(max(y_offsets)))  # Exclusive
    print('idx min & max:', min_t, max_t)
    idx = np.arange(min_t, max_t, 1)
    return data, idx


folder_path = "WEATHER-5K/global_weather_stations"
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

def generate_train_val_test(args):
    years = args['years'].split('_')
    max_wind_rate = 0
    for y in years:
        year_data = []
        for file in csv_files:
            file_path = os.path.join(folder_path, file)
            df = pd.read_csv(file_path)
            df['DATE'] = pd.to_datetime(df['DATE'])
            df = df[df['DATE'].dt.year == int(y)]
            max_wind_rate = max(max_wind_rate, df['WND_RATE'].max())
            df_value = df.values
            year_data.append(df_value)

        stacked_year_data = np.stack(year_data, axis=1)

    print("max_wind_rate",max_wind_rate)
    print('original data shape:', stacked_year_data.shape)
    print("original columns:", df.columns)
    seq_length_x, seq_length_y = args['seq_length_x'], args['seq_length_y']
    x_offsets = np.arange(-(seq_length_x - 1), 1, 1)
    y_offsets = np.arange(1, (seq_length_y + 1), 1)

    data, idx = generate_data_and_idx(stacked_year_data, x_offsets, y_offsets, args['tod'], args['dow'])
    
    # 原本的列 ['DATE', 'LONGITUDE', 'LATITUDE', 'TMP', 'DEW', 'WND_ANGLE', 'WND_RATE','SLP', 'MASK', 'TIME_DIFF']
    col_to_use = ['TMP', 'DEW', 'WND_ANGLE', 'WND_RATE', 'SLP', 'MASK']
    data = data[:, :, [df.columns.get_loc(c) for c in col_to_use]]
    # print(data)
    print('final data shape:', data.shape, 'idx shape:', idx.shape)
    
    num_samples = len(idx)
    num_train = round(num_samples * 0.6)
    num_val = round(num_samples * 0.2)   

    # split idx
    idx_train = idx[:num_train]
    idx_val = idx[num_train: num_train + num_val]
    idx_test = idx[num_train + num_val:]
    
    #
    normalize_col = ['TMP','DEW','SLP'] # 对应 0,1,4
    max_normlize = ['WND_ANGLE','WND_RATE'] # 对应 2,3

    # normalize
    for row in [0,1,4]:
        x_train = data[:idx_val[0] - args['seq_length_x'], :, row] 
        scaler = StandardScaler(mean=x_train.mean(), std=x_train.std())
        data[..., row] = scaler.transform(data[..., row])

    data[..., 2] = data[..., 2] / 360.0
    data[..., 3] = data[..., 3] / max_wind_rate


    # save
    out_dir = args['dataset'] + '/' + args['years']
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
    np.savez_compressed(os.path.join(out_dir, 'his.npz'), data=data, mean=scaler.mean, std=scaler.std)

    np.save(os.path.join(out_dir, 'idx_train'), idx_train)
    np.save(os.path.join(out_dir, 'idx_val'), idx_val)
    np.save(os.path.join(out_dir, 'idx_test'), idx_test)

arg = {
    'dataset': 'weather',
    'years': '2023',
    'seq_length_x': 12,
    'seq_length_y': 12,
    'tod': 1,
    'dow': 1
}
generate_train_val_test(arg)

max_wind_rate 24.7
original data shape: (8760, 10, 10)
original columns: Index(['DATE', 'LONGITUDE', 'LATITUDE', 'TMP', 'DEW', 'WND_ANGLE', 'WND_RATE',
       'SLP', 'MASK', 'TIME_DIFF'],
      dtype='object')
num_samples: 8760 num_nodes: 10
idx min & max: 11 8748
final data shape: (8760, 10, 6) idx shape: (8737,)


查看npz

In [ ]:
import numpy as np

# 读取 .npz 文件
file_path = 'weather/2023/his.npz'  # 替换为你的 .npz 文件路径
data = np.load(file_path, allow_pickle=True)

# 查看 .npz 文件中包含的数组名称
print("Arrays in the .npz file:", data.files)


in_data = data['data']
in_mean = data['mean']
in_std = data['std']

print("Data shape:", in_data.shape)

print("Mean shape:", in_mean.shape)
print(in_mean)
print("Std shape:", in_std.shape)
print(in_std)

# 关闭文件（重要！）
data.close()

Arrays in the .npz file: ['data', 'mean', 'std']
Data shape: (8760, 10, 6)
-0.005590526966527427
0.9704709305012484
Mean shape: ()
1008.6588885963583
Std shape: ()
12.399915678501442


存储成H5文件

In [18]:
import pandas as pd
import numpy as np
import os
import h5py
folder_path = "WEATHER-5K/global_weather_stations"
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
# mask代表数据的缺失情况
year_data = []
columns = ['DATE','LONGITUDE','LATITUDE','TMP','DEW','WND_ANGLE','WND_RATE','SLP','MASK','TIME_DIFF']
columns_2 = ['DATE','LONGITUDE','LATITUDE','TMP','DEW','WND_ANGLE','WND_RATE','SLP','MASK_2','TIME_DIFF']
def mask_to_binary(mask_str):
    # 去掉首尾的括号
    mask_str = mask_str.strip('[]')
    # 将字符串分割成列表
    mask_list = mask_str.split()
    # 将列表中的字符串转换为整数
    binary_numbers = [int(num) for num in mask_list]
    # 将整数列表转换为二进制数
    binary_str = ''.join(map(str, binary_numbers))
    return int(binary_str, 2)

for year in range(2014,3000):
    for file in csv_files:
        file_path = os.path.join(folder_path, file)
        df = pd.read_csv(file_path)
        df['DATE'] = pd.to_datetime(df['DATE'])
        df = df[df['DATE'].dt.year == year]
        df = df[columns]
        df['MASK_2'] = df['MASK'].apply(mask_to_binary)
        df = df[columns_2]

        df_value = df.iloc[:,1:].values
        
        year_data.append(df_value)
        # break
    # print(df['DATE'].dt.year)
    

    stacked_year_data = np.stack(year_data, axis=1)
    # print(stacked_year_data.shape)
    # print(stacked_year_data[0][0])

    # 创建一个名为 data 的数据集，数据集的内容就是我们的数据
    file_path = f'weather_his_{year}.h5'
    with h5py.File(file_path, 'a') as h5_file:
        dataset_name = 'data'
        
        # 如果数据集存在，先删除它
        if dataset_name in h5_file:
            del h5_file[dataset_name]
        
        # 创建新的数据集
        h5_file.create_dataset(dataset_name, data=stacked_year_data)
        h5_file[dataset_name].attrs['columns'] = columns_2
    break


#   ca_his_2019.h5

# print(df_year.shape)
# print(df_year.head(5))

In [19]:
file_path = 'weather_his_2014.h5'

# 打开 .h5 文件
with h5py.File(file_path, 'r') as h5_file:
    # 读取数据集
    print(h5_file.keys())
    print(list(h5_file[dataset_name].attrs['columns']))
    dataset_name = 'data'  # 数据集的名称
    if dataset_name in h5_file:
        data = h5_file[dataset_name][:]
        print(f"数据形状: {data.shape}")
    else:
        print(f"数据集 '{dataset_name}' 不存在于文件中。")
    
    # print(data.keys)
    print(data[0][0])

<KeysViewHDF5 ['data']>
['DATE', 'LONGITUDE', 'LATITUDE', 'TMP', 'DEW', 'WND_ANGLE', 'WND_RATE', 'SLP', 'MASK_2', 'TIME_DIFF']
数据形状: (8760, 10, 9)
[-8.66666670e+00  7.09333333e+01 -8.00000000e-01 -4.00000000e+00
  2.90000000e+02  6.00000000e+00  1.01220000e+03  3.10000000e+01
  0.00000000e+00]


In [3]:
import pandas as pd
import numpy as np
import os
import h5py
from tqdm import tqdm  # 可选进度条

folder_path = "WEATHER-5K/global_weather_stations"
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
columns = ['DATE','LONGITUDE','LATITUDE','TMP','DEW','WND_ANGLE','WND_RATE','SLP','MASK','TIME_DIFF']

# 修正年份范围（2014-3000不合理）
start_year = 2014
end_year = 2020  # 调整为实际需要的结束年份

# 定义h5py变长字符串类型
vlen_str = h5py.special_dtype(vlen=str)

for year in tqdm(range(start_year, end_year+1)):
    yearly_data = []
    station_ids = []
    
    for file in csv_files:
        try:
            file_path = os.path.join(folder_path, file)
            df = pd.read_csv(file_path)
            
            # 提取站点ID（假设文件名包含站点信息）
            station_id = os.path.splitext(file)[0]
            
            # 转换日期并过滤年份
            df['DATE'] = pd.to_datetime(df['DATE'])
            df_year = df[df['DATE'].dt.year == year]
            
            if not df_year.empty:
                # 确保列存在
                df_year = df_year[columns].copy()
                
                # 转换MASK列为字符串列表
                df_year['MASK'] = df_year['MASK'].astype(str).tolist()
                
                # 存储站点ID和数据
                station_ids.append(station_id)
                yearly_data.append(df_year.values)
                
        except Exception as e:
            print(f"Error processing {file} for year {year}: {str(e)}")
            continue

    if yearly_data:
        # 创建年度HDF5文件
        with h5py.File(f'1weather_his_{year}.h5', 'w') as hf:
            # 存储站点ID作为属性
            hf.attrs['station_ids'] = station_ids
            
            # 创建复合数据类型
            dt = np.dtype([
                ('DATE', 'datetime64[s]'),
                ('LONGITUDE', 'f4'),
                ('LATITUDE', 'f4'),
                ('TMP', 'f4'),
                ('DEW', 'f4'),
                ('WND_ANGLE', 'f4'),
                ('WND_RATE', 'f4'),
                ('SLP', 'f4'),
                ('MASK', vlen_str),
                ('TIME_DIFF', 'f4')
            ])
            
            # 转换数据
            stacked_data = np.concatenate(yearly_data)
            structured_data = np.array(
                [tuple(row) for row in stacked_data],
                dtype=dt
            )
            
            # 创建数据集
            hf.create_dataset('weather_data', data=structured_data)
            
            # 添加元数据
            hf['weather_data'].attrs['columns'] = columns
            hf['weather_data'].attrs['year'] = year
    else:
        print(f"No data found for year {year}")

print("Processing completed.")

  0%|          | 0/7 [00:01<?, ?it/s]


TypeError: No conversion path for dtype: dtype('<M8[s]')